---

### Finite Automata Explainer

<pre>
Burton Rosenberg
19 January 2025
University of Miami
</pre>

---

These are some small code sketches to explain the idea of a Finite Automata.

A Finite Automata is a machine that can the membership problem for a _language_, that is, the subset of the set of all words over an alphabet,

$$
S \subseteq \Sigma^*
$$

Let us suppose a two letter alphabet $\Sigma = \{\,a,b\,\}$, and the language $S$ to be all strings that contain at least one $a$. 


### Step 1: Write a program

__Exercise 1:__ Write a python program that returns `True` when the given string `s` is in `S`; and returns `False` otherwise.



In [1]:
# exercise 1

def member_at_least_one(s):
    saw_an_a = 'no'
    for letter in s:
        if letter=='a':
            saw_an_a = 'yes'
    return saw_an_a=='yes'

examples = ['a', 'bab', 'babab','', 'b', 'bbb']

for s in examples:
    print(f'M(|{s}|) = {member_at_least_one(s)}')

M(|a|) = True
M(|bab|) = True
M(|babab|) = True
M(||) = False
M(|b|) = False
M(|bbb|) = False


### Step 2: The essence of the idea

There maybe otherways to write this code, but I wanted a plausible code that illustrates a possible and general solution to this and similar word recognition problems.

1. The code initializes some variables, 
1. The code examines letters in the word one by one,
1. The code updates the variables on examining each letter,
1. When all letters have been examined, the code returns true or false based on the final values of the variables.

Let us generalize the above code. 

1. Have a `state` variable take on several values.
1. Let `state` be initialzed to the state `q0`.
1. A `transitions` map will update `state` based on both its current value and the letter read from the word.
1. After all letters have in turn updated `state`, the progra returns `True` if the final value is in the set `accept`.

The `transitions` map is the most involved item on this list, and is the heart of what the program will compute. It needs to record the direction,

- If $b$ is seen, stay in the current state.
- If $a$ is seen, transtion to the `q1` state.

__Exercise 2:__ Rewrite the above python program to accept $S$, using the method just described.


In [2]:
# exercise 2

def member_at_least_one_FA(s):
    state = 'q0'
    accept = {'q1'}  # the set() constructor works differently
    transitions = {
        ('q0','b'): 'q0',
        ('q1','b'): 'q1',
        ('q0','a'): 'q1',
        ('q1','a'): 'q1',
    }
    for letter in s:
        state = transitions[(state,letter)]
    return state in accept

examples = ['a', 'bab', 'babab','', 'b', 'bbb']

for s in examples:
    print(f'M(|{s}|) = {member_at_least_one_FA(s)}')

M(|a|) = True
M(|bab|) = True
M(|babab|) = True
M(||) = False
M(|b|) = False
M(|bbb|) = False



Why would on go through all the trouble to rewrite this code this way? Because now just by changing the _data_ (the transitions function, the start state, the accept states) the function is changed. Allowing us to study the _machine_, considering its action under all possible programmings.

__Exercise 3:__ Modify just that data in the previous program to now accept the language of all strings with 2 or more $a$'s.


In [3]:
# exercise 3

def member_at_least_two_FA(s):
    state = 'q0'
    accept = {'q2'}
    transitions = {
        ('q0','b'): 'q0',
        ('q1','b'): 'q1',
        ('q2','b'): 'q2',
        ('q0','a'): 'q1',
        ('q1','a'): 'q2',
        ('q2','a'): 'q2',
    }
    for letter in s:
        state = transitions[(state,letter)]
    return state in accept

examples = ['a', 'bab', 'babab','', 'b', 'bbb']

for s in examples:
    print(f'M(|{s}|) = {member_at_least_two_FA(s)}')

M(|a|) = False
M(|bab|) = False
M(|babab|) = True
M(||) = False
M(|b|) = False
M(|bbb|) = False


### Step 3: A Finite Automata Object

The last transformation I want to do, is to make the division of data and programming more defined. We use Object Oriented Programming, as OOP is about centering our programming on the data, not the process.

The data is collected into a map, with fields `start`, `transitions`, `accept`, as before, as now also includes `alphabet` and `states`, so that the acceptable space of values is defined.

The length of this code is due to error checking and `verbose` handling. It is really as simple as the previous code.


In [4]:
#
# definition of class SimpleFiniteAutomata
# for csc427 semester 232 (jan 2023-may 2023)
# last-update:
#      22 jan 2023 -bjr: created
#      23 jan -bjr: added assertion handling
#


class SimpleFiniteAutomata:
    
    def __init__(self,fa_description,verbose=False):
        self.fa = fa_description
        self.state = self.fa['start']
        self.verbose = verbose
        
    def one_step(self,symbol):
        assert symbol in self.fa['alphabet'] , f"Symbol |{symbol}| not in the alphabet."
        assert (self.state,symbol) in self.fa['transitions'] , f"Transition |{(self.state,symbol)}| undefined."
        return self.fa['transitions'][(self.state,symbol)]
        
    def compute(self,string):
        self.state = self.fa['start']
        if self.verbose:
            print(f'\ninput: |{string}|')
        for symbol in string:
            s = self.one_step(symbol)
            if self.verbose:
                print(f'({self.state},{symbol}) -> {s}')
            self.state = s
        if self.verbose:
            s = ('reject','accept')[self.state in self.fa['accept']]
            print(s)
        return self.state in self.fa['accept']

    
#end class SimpleFiniteAutomata



__Exercise 4:__ Write the machine descriptions for the two machines, at least one $a$ and at least two $a$'s and test.


In [5]:

at_least_one = {
    'alphabet': {'a','b'},
    'states': {'q0','q1'},
    'start':'q0',
    'accept':{'q1'},
    'transitions':{
        ('q0','b'): 'q0',
        ('q1','b'): 'q1',
        ('q0','a'): 'q1',
        ('q1','a'): 'q1',
    } 
}

at_least_two = {
    'alphabet': {'a','b'},
    'states': {'q0','q1','q2'},
    'start':'q0',
    'accept':{'q2'},
    'transitions':{
        ('q0','b'): 'q0',
        ('q1','b'): 'q1',
        ('q2','b'): 'q2',
        ('q0','a'): 'q1',
        ('q1','a'): 'q2',
        ('q2','a'): 'q2',
    }   
}

sfa = SimpleFiniteAutomata(at_least_one)
print(f'\nAt least one\n------------')
for s in examples:
    print(f'M(|{s}|) = {sfa.compute(s)}')

print(f'\nAt least two\n------------')
sfa = SimpleFiniteAutomata(at_least_two)
for s in examples:
    print(f'M(|{s}|) = {sfa.compute(s)}')


At least one
------------
M(|a|) = True
M(|bab|) = True
M(|babab|) = True
M(||) = False
M(|b|) = False
M(|bbb|) = False

At least two
------------
M(|a|) = False
M(|bab|) = False
M(|babab|) = True
M(||) = False
M(|b|) = False
M(|bbb|) = False


In [6]:
# end